In [1]:
import nltk
# nltk.download()

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import csv
import json

In [3]:
#read in csv
types = pd.read_csv("../mbti_1.csv")
types.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [4]:
# types['posts'].iloc[0]

# EDA

In [5]:
types.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8675 entries, 0 to 8674
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   type    8675 non-null   object
 1   posts   8675 non-null   object
dtypes: object(2)
memory usage: 135.7+ KB


In [6]:
# check for null
types.isnull().sum()

type     0
posts    0
dtype: int64

In [7]:
# unique check 
types.nunique()

type       16
posts    8675
dtype: int64

In [8]:
types.type.value_counts()

INFP    1832
INFJ    1470
INTP    1304
INTJ    1091
ENTP     685
ENFP     675
ISTP     337
ISFP     271
ENTJ     231
ISTJ     205
ENFJ     190
ISFJ     166
ESTP      89
ESFP      48
ESFJ      42
ESTJ      39
Name: type, dtype: int64

In [9]:
# add plots

# Preprocessing

In [10]:
# removing piping
types['posts']= types['posts'].str.replace('|',' ')
types.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw ...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired. That's another silly misconce...


In [11]:
# removing '
types['posts']= types['posts'].str.replace("'",'')
types.head()

,type,posts
0,INFJ,http://www.youtube.com/watch?v=qsXHcwe3krw h...
1,ENTP,Im finding the lack of me in these posts very ...
2,INTP,Good one _____ https://www.youtube.com/watc...
3,INTJ,"Dear INTP, I enjoyed our conversation the ot..."
4,ENTJ,Youre fired. Thats another silly misconcepti...


In [12]:
# removing url's from posts
types['posts'] = types['posts'].str.replace('http\S+|www.\S+', '', case=False)
types.head()

,type,posts
0,INFJ,enfp and intj moments sportscenter no...
1,ENTP,Im finding the lack of me in these posts very ...
2,INTP,"Good one _____ Of course, to which I say..."
3,INTJ,"Dear INTP, I enjoyed our conversation the ot..."
4,ENTJ,Youre fired. Thats another silly misconcepti...


In [13]:
# change case to lower
types['posts'] = types['posts'].str.lower()
types.head()

,type,posts
0,INFJ,enfp and intj moments sportscenter no...
1,ENTP,im finding the lack of me in these posts very ...
2,INTP,"good one _____ of course, to which i say..."
3,INTJ,"dear intp, i enjoyed our conversation the ot..."
4,ENTJ,youre fired. thats another silly misconcepti...


In [14]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [15]:
#remove punctuation from posts

def remove_punctuation(text):
    no_punctuation = "".join([char for char in text if char not in string.punctuation])
    return no_punctuation

types['body_text_clean'] = types['posts'].apply(lambda x: remove_punctuation(x))


In [16]:
# pulling types from type column
mbti_types = types['type'].unique()

In [17]:
# types to list instead of array
mbti_list = mbti_types.tolist()

In [18]:
# lowercasing types
mbti_new = [x.lower() for x in mbti_list]

In [19]:
# remove references to mbti type in body_text_clean column

for item in mbti_new:
    types['body_text_clean'] = types['body_text_clean'].str.replace(item , "")

types.head()

,type,posts,body_text_clean
0,INFJ,enfp and intj moments sportscenter no...,and moments sportscenter not top te...
1,ENTP,im finding the lack of me in these posts very ...,im finding the lack of me in these posts very ...
2,INTP,"good one _____ of course, to which i say...",good one of course to which i say i kno...
3,INTJ,"dear intp, i enjoyed our conversation the ot...",dear i enjoyed our conversation the other d...
4,ENTJ,youre fired. thats another silly misconcepti...,youre fired thats another silly misconceptio...


In [20]:
# apply word_tokenize to all records
from nltk.tokenize import word_tokenize

types['tokenized'] = types['body_text_clean'].apply(word_tokenize)

types.head()

,type,posts,body_text_clean,tokenized
0,INFJ,enfp and intj moments sportscenter no...,and moments sportscenter not top te...,"[and, moments, sportscenter, not, top, ten, pl..."
1,ENTP,im finding the lack of me in these posts very ...,im finding the lack of me in these posts very ...,"[im, finding, the, lack, of, me, in, these, po..."
2,INTP,"good one _____ of course, to which i say...",good one of course to which i say i kno...,"[good, one, of, course, to, which, i, say, i, ..."
3,INTJ,"dear intp, i enjoyed our conversation the ot...",dear i enjoyed our conversation the other d...,"[dear, i, enjoyed, our, conversation, the, oth..."
4,ENTJ,youre fired. thats another silly misconcepti...,youre fired thats another silly misconceptio...,"[youre, fired, thats, another, silly, misconce..."


In [21]:
# types['body_text_clean'].iloc[0]

In [22]:
# remove stopwords
from nltk.corpus import stopwords
stop = stopwords.words('english')


def stopword_removal(text):
    stop_words = [item for item in text if item not in stop]
    return stop_words

types['stopwords'] = types['tokenized'].apply(lambda x: stopword_removal(x))

In [23]:
# stem porter
import nltk
from nltk.stem import PorterStemmer
ps = nltk.PorterStemmer()

def porter_stem(text):
    porter = [ps.stem(word) for word in text]
    return porter

types['porter_stem'] = types['stopwords'].apply(lambda x: porter_stem(x))

In [24]:
types.head()

,type,posts,body_text_clean,tokenized,stopwords,porter_stem
0,INFJ,enfp and intj moments sportscenter no...,and moments sportscenter not top te...,"[and, moments, sportscenter, not, top, ten, pl...","[moments, sportscenter, top, ten, plays, prank...","[moment, sportscent, top, ten, play, prank, li..."
1,ENTP,im finding the lack of me in these posts very ...,im finding the lack of me in these posts very ...,"[im, finding, the, lack, of, me, in, these, po...","[im, finding, lack, posts, alarming, sex, bori...","[im, find, lack, post, alarm, sex, bore, posit..."
2,INTP,"good one _____ of course, to which i say...",good one of course to which i say i kno...,"[good, one, of, course, to, which, i, say, i, ...","[good, one, course, say, know, thats, blessing...","[good, one, cours, say, know, that, bless, cur..."
3,INTJ,"dear intp, i enjoyed our conversation the ot...",dear i enjoyed our conversation the other d...,"[dear, i, enjoyed, our, conversation, the, oth...","[dear, enjoyed, conversation, day, esoteric, g...","[dear, enjoy, convers, day, esoter, gab, natur..."
4,ENTJ,youre fired. thats another silly misconcepti...,youre fired thats another silly misconceptio...,"[youre, fired, thats, another, silly, misconce...","[youre, fired, thats, another, silly, misconce...","[your, fire, that, anoth, silli, misconcept, a..."


In [25]:
# stem snowball
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

def snowball(text):
    snow = [stemmer.stem(y) for y in text]
    return snow

types['snow_stem'] = types['stopwords'].apply(lambda x: snowball(x))
types.head()

,type,posts,body_text_clean,tokenized,stopwords,porter_stem,snow_stem
0,INFJ,enfp and intj moments sportscenter no...,and moments sportscenter not top te...,"[and, moments, sportscenter, not, top, ten, pl...","[moments, sportscenter, top, ten, plays, prank...","[moment, sportscent, top, ten, play, prank, li...","[moment, sportscent, top, ten, play, prank, li..."
1,ENTP,im finding the lack of me in these posts very ...,im finding the lack of me in these posts very ...,"[im, finding, the, lack, of, me, in, these, po...","[im, finding, lack, posts, alarming, sex, bori...","[im, find, lack, post, alarm, sex, bore, posit...","[im, find, lack, post, alarm, sex, bore, posit..."
2,INTP,"good one _____ of course, to which i say...",good one of course to which i say i kno...,"[good, one, of, course, to, which, i, say, i, ...","[good, one, course, say, know, thats, blessing...","[good, one, cours, say, know, that, bless, cur...","[good, one, cours, say, know, that, bless, cur..."
3,INTJ,"dear intp, i enjoyed our conversation the ot...",dear i enjoyed our conversation the other d...,"[dear, i, enjoyed, our, conversation, the, oth...","[dear, enjoyed, conversation, day, esoteric, g...","[dear, enjoy, convers, day, esoter, gab, natur...","[dear, enjoy, convers, day, esoter, gab, natur..."
4,ENTJ,youre fired. thats another silly misconcepti...,youre fired thats another silly misconceptio...,"[youre, fired, thats, another, silly, misconce...","[youre, fired, thats, another, silly, misconce...","[your, fire, that, anoth, silli, misconcept, a...","[your, fire, that, anoth, silli, misconcept, a..."


In [26]:
types.tail()

,type,posts,body_text_clean,tokenized,stopwords,porter_stem,snow_stem
8670,ISFP,ixfp just because i always think of cats as...,ixfp just because i always think of cats as...,"[ixfp, just, because, i, always, think, of, ca...","[ixfp, always, think, cats, fi, doms, reason, ...","[ixfp, alway, think, cat, fi, dom, reason, esp...","[ixfp, alway, think, cat, fi, dom, reason, esp..."
8671,ENFP,so...if this thread already exists someplace e...,soif this thread already exists someplace else...,"[soif, this, thread, already, exists, someplac...","[soif, thread, already, exists, someplace, els...","[soif, thread, alreadi, exist, someplac, els, ...","[soif, thread, alreadi, exist, someplac, els, ..."
8672,INTP,so many questions when i do these things. i w...,so many questions when i do these things i wo...,"[so, many, questions, when, i, do, these, thin...","[many, questions, things, would, take, purple,...","[mani, question, thing, would, take, purpl, pi...","[mani, question, thing, would, take, purpl, pi..."
8673,INFP,i am very conflicted right now when it comes t...,i am very conflicted right now when it comes t...,"[i, am, very, conflicted, right, now, when, it...","[conflicted, right, comes, wanting, children, ...","[conflict, right, come, want, children, honest...","[conflict, right, come, want, children, honest..."
8674,INFP,it has been too long since i have been on pers...,it has been too long since i have been on pers...,"[it, has, been, too, long, since, i, have, bee...","[long, since, personalitycafe, although, doesn...","[long, sinc, personalitycaf, although, doesnt,...","[long, sinc, personalitycaf, although, doesnt,..."


In [27]:
# lemmatize
from nltk.stem import WordNetLemmatizer 
  
lm = WordNetLemmatizer()

def lemma_words(lemma):
    lemmatize = [lm.lemmatize(word) for word in lemma]
    return lemmatize

types['lemmatized'] = types['stopwords'].apply(lambda x: lemma_words(x))
types.head()

,type,posts,body_text_clean,tokenized,stopwords,porter_stem,snow_stem,lemmatized
0,INFJ,enfp and intj moments sportscenter no...,and moments sportscenter not top te...,"[and, moments, sportscenter, not, top, ten, pl...","[moments, sportscenter, top, ten, plays, prank...","[moment, sportscent, top, ten, play, prank, li...","[moment, sportscent, top, ten, play, prank, li...","[moment, sportscenter, top, ten, play, prank, ..."
1,ENTP,im finding the lack of me in these posts very ...,im finding the lack of me in these posts very ...,"[im, finding, the, lack, of, me, in, these, po...","[im, finding, lack, posts, alarming, sex, bori...","[im, find, lack, post, alarm, sex, bore, posit...","[im, find, lack, post, alarm, sex, bore, posit...","[im, finding, lack, post, alarming, sex, borin..."
2,INTP,"good one _____ of course, to which i say...",good one of course to which i say i kno...,"[good, one, of, course, to, which, i, say, i, ...","[good, one, course, say, know, thats, blessing...","[good, one, cours, say, know, that, bless, cur...","[good, one, cours, say, know, that, bless, cur...","[good, one, course, say, know, thats, blessing..."
3,INTJ,"dear intp, i enjoyed our conversation the ot...",dear i enjoyed our conversation the other d...,"[dear, i, enjoyed, our, conversation, the, oth...","[dear, enjoyed, conversation, day, esoteric, g...","[dear, enjoy, convers, day, esoter, gab, natur...","[dear, enjoy, convers, day, esoter, gab, natur...","[dear, enjoyed, conversation, day, esoteric, g..."
4,ENTJ,youre fired. thats another silly misconcepti...,youre fired thats another silly misconceptio...,"[youre, fired, thats, another, silly, misconce...","[youre, fired, thats, another, silly, misconce...","[your, fire, that, anoth, silli, misconcept, a...","[your, fire, that, anoth, silli, misconcept, a...","[youre, fired, thats, another, silly, misconce..."


In [28]:
# types.to_csv('preprocess.csv')

# Vectorizing

In [29]:
# Count Vectorizing
import re

def clean_posts(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    post = re.split('\W+', text)
    text = [ps.stem(word) for word in post if word not in nltk.corpus.stopwords.words('english')]
    return text

In [30]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorize = CountVectorizer(analyzer=clean_posts)
vectorized = count_vectorize.fit_transform(types['posts'])

In [31]:
print(vectorized.shape)
print(count_vectorize.get_feature_names())

(8675, 127076)
['', '0', '00', '000', '0000', '000000', '000000000000000', '00000000001', '00000011', '000000111', '0000001111', '00001010', '0001', '00023768', '000400', '001', '00100000', '00100001', '00100111', '001005', '00101100', '00101110', '00111111', '001311', '00159265', '002', '003', '004', '004200', '005', '006', '0060', '007', '007phantom', '009', '00earli', '00hikaru00', '00searli', '01', '010', '0100', '01000010', '01001000', '01001001', '0100101000101010010', '01001100', '0100110101100001011110010110001001100101', '01001110', '01010000', '01010011', '010101', '01010100', '01010111', '01011001', '010337', '011', '01100001', '01100010', '01100011', '01100100', '01100101', '01100110', '01100111', '01101000', '01101001', '01101011', '01101100', '01101100011011110110110000101110', '01101101', '01101110', '01101111', '01110000', '01110010', '01110011', '01110100', '01110101', '01110110', '01110111', '01111', '01111000', '01111001', '012', '013', '014', '0144455323362841', '01

In [ ]:
# N-Gram Vectorizing

ngram_vect = CountVectorizer(ngram_range=(2,2), analyzer=clean_posts)
X_counts = ngram_vect.fit_transform(types['posts'])
print(X_counts.shape)
print(ngram_vect.get_feature_names())